In [1]:
import brainsss
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
import random
from scipy.stats import sem
import time
import h5py
import ants
import nibabel as nib
import matplotlib
import pickle
from shutil import copyfile
from mpl_toolkits.axes_grid1 import ImageGrid
from skimage.filters import threshold_otsu as otsu
from scipy.ndimage.morphology import binary_erosion
from scipy.ndimage.morphology import binary_dilation
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap

/home/users/brezovec/.local/lib/python3.6/site-packages/ants/viz/render_surface_function.py:16: UserWarning: Cant import Plotly. Install it `pip install chart_studio` if you want to use ants.render_surface_function
  "Cant import Plotly. Install it `pip install chart_studio` if you want to use ants.render_surface_function"


In [2]:
### DEFINE DIRECTORIES ###
oak = "/oak/stanford/groups/trc/data/"
warp_types = ['direct_affine', 'direct_syn', 'localmean_ANTS', 'localmean_synthmorph']

directories_clandinin = {
     'direct_affine': "Brezovec/2P_Imaging/20221029_FDA_direct_affine/clandinin",
     'direct_syn': "Brezovec/2P_Imaging/20221029_FDA_direct_syn/clandinin",
     'localmean_ANTS': "Alex/clab_data/LC11/func/aligned_to_FDA/",
     'localmean_synthmorph': "Alex/clab_data/LC11/func/synthmorph_to_FDA"
}

for warp_type in warp_types:
    directories_clandinin[warp_type] = os.path.join(oak, directories_clandinin[warp_type])
    
directories_murthy = {
     'direct_affine': "Brezovec/2P_Imaging/20221029_FDA_direct_affine/murthy",
     'direct_syn': "Brezovec/2P_Imaging/20221029_FDA_direct_syn/murthy",
     'localmean_ANTS': "Alex/albert_data/LC11/func/aligned_to_FDA/",
     'localmean_synthmorph': "Alex/albert_data/LC11/func/synthmorph_to_FDA"
}
    
for warp_type in warp_types:
    directories_murthy[warp_type] = os.path.join(oak, directories_murthy[warp_type])
    

In [3]:
###########################################
### Load warped indiv LC11 gloms in FDA ###
###########################################

######################################
### HERE IS KEY VARIABLE TO CHANGE ###
warp_type = 'direct_affine'
######################################

brains = {}
brains_cropped = {}
brain_types = ['clandinin', 'murthy', 'clandinin_flip', 'murthy_flip', 'connectome']
for brain_type in brain_types:
    if 'clandinin' in brain_type:
        dir_ = directories_clandinin[warp_type]
    elif 'murthy' in brain_type:
        dir_ = directories_murthy[warp_type]
    elif 'connectome' in brain_type:
        LC11_fda_path = "/oak/stanford/groups/trc/data/Alex/clab_data/LC11/LC11_FDA_ds_flip.nii"
        LC11_fda = ants.image_read(LC11_fda_path)[:]
        LC11_fda_cropped = LC11_fda[bbox['x_start']:bbox['x_end'],
                           bbox['y_start']:bbox['y_end'],
                           bbox['z_start']:bbox['z_end']]
        brains[brain_type] = [LC11_fda]
        brains_cropped[brain_type] = [LC11_fda_cropped]
        continue
        
    brains[brain_type] = []
    files = [x for x in os.listdir(dir_) if 'red' in x] ### just need to switch this for localmean_synthmorph
    #files = [x for x in os.listdir(dir_)]
    for file in files:
        print(file)
        to_load = os.path.join(dir_,file)
        image = ants.image_read(to_load)
        
        if 'flip' in brain_type:
            image = image[::-1,:,:]
        
        brains[brain_type].append(image)

210223_04_red-to-FDA.nii
210208_02_red-to-FDA.nii
210223_03_red-to-FDA.nii
210216_04_red-to-FDA.nii
210216_02_red-to-FDA.nii
210208_03_red-to-FDA.nii
210218_02_red-to-FDA.nii
210223_02_red-to-FDA.nii
210223_01_red-to-FDA.nii
220412_LC11_vol1_local_atlas_red-to-FDA.nii
220420_LC11_vol2_local_atlas_red-to-FDA.nii
220426_LC11_vol1_local_atlas_red-to-FDA.nii
220413_LC11_vol1_local_atlas_red-to-FDA.nii
220425_LC11_vol1_local_atlas_red-to-FDA.nii
220420_LC11_vol1_local_atlas_red-to-FDA.nii
220426_LC11_vol2_local_atlas_red-to-FDA.nii
220419_LC11_vol2_local_atlas_red-to-FDA.nii
210223_04_red-to-FDA.nii
210208_02_red-to-FDA.nii
210223_03_red-to-FDA.nii
210216_04_red-to-FDA.nii
210216_02_red-to-FDA.nii
210208_03_red-to-FDA.nii
210218_02_red-to-FDA.nii
210223_02_red-to-FDA.nii
210223_01_red-to-FDA.nii
220412_LC11_vol1_local_atlas_red-to-FDA.nii
220420_LC11_vol2_local_atlas_red-to-FDA.nii
220426_LC11_vol1_local_atlas_red-to-FDA.nii
220413_LC11_vol1_local_atlas_red-to-FDA.nii
220425_LC11_vol1_local

NameError: name 'bbox' is not defined